# Implementation of the solution to project 2 from scratch
To note:
- For clarity the df_test was renamed to df_val as the test word was used when splitting the labeled data into train and test. 
    - Val stands for validation

To try out:
- Preprocessing
    - Tweak data imputer
    - Tweak scaler (Robust scaler, minmax, etc..)
    - Tweak feature selection parameter
    - Tweak order of operations above to see the effect
- Modelling
    - XGBoost
    - SVM

## Import modules

In [152]:
import argparse
import logging
import os
import shutil
import sys
import zipfile
import time
import sys
import torch

import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter
from imblearn.over_sampling import ADASYN, SMOTE
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler

from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.feature_selection import SelectKBest, f_regression, chi2, f_classif
from sklearn.metrics import f1_score, mean_squared_error, accuracy_score, r2_score, roc_auc_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import label_ranking_average_precision_score as LRAPS
from sklearn.metrics import label_ranking_loss as LRL
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, BayesianRidge, LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

## Define global variables

In [153]:
# Global variables
IDENTIFIERS = ["pid", "Time"]
MEDICAL_TESTS = [
    "LABEL_BaseExcess",
    "LABEL_Fibrinogen",
    "LABEL_AST",
    "LABEL_Alkalinephos",
    "LABEL_Bilirubin_total",
    "LABEL_Lactate",
    "LABEL_TroponinI",
    "LABEL_SaO2",
    "LABEL_Bilirubin_direct",
    "LABEL_EtCO2",
]
VITAL_SIGNS = ["LABEL_RRate", "LABEL_ABPm", "LABEL_SpO2", "LABEL_Heartrate"]
SEPSIS = ["LABEL_Sepsis"]
ESTIMATOR = {"bayesian": BayesianRidge(), "decisiontree": DecisionTreeRegressor(max_features="sqrt", random_state=0), 
                "extratree": ExtraTreesRegressor(n_estimators=10, random_state=0), 
                "knn": KNeighborsRegressor(n_neighbors=10, weights="distance")}
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [154]:
def sigmoid_f(x):
    """To get predictions as confidence level, the model predicts for all 12 sets of measures for
    each patient a distance to the hyperplane ; it is then transformed into a confidence level using
    the sigmoid function ; the confidence level reported is the mean of all confidence levels for a
    single patient

    Args:
        x (float): input of the sigmoid function

    Returns:
       float: result of the sigmoid computation.

    """
    return 1 / (1 + np.exp(-x))

## Load Data

In [155]:
df_train = pd.read_csv(r"data/train_features.csv")
df_train_label = pd.read_csv(r"data/train_labels.csv")
df_val = pd.read_csv(r"data/test_features.csv")

## Data imputation methodology

In [156]:
# Fit imputer to missing data
pid_train = df_train["pid"].unique()
columns = df_train.columns
df_train_preprocessed = pd.DataFrame(columns=columns, index=pid_train)

imputer = IterativeImputer()
columns = df_train.columns
df_train = imputer.fit_transform(df_train.values)
df_train = pd.DataFrame(df_train, columns=columns)
df_train = df_train.groupby(["pid"], as_index=False).mean()

/home/philiphartout/anaconda3/envs/iml/lib/python3.7/site-packages/sklearn/impute/_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [157]:
# Tranform test data according to same imputer
pid_val = df_val["pid"].unique()
columns = df_val.columns
df_val_preprocessed = pd.DataFrame(columns=columns, index=pid_val)

df_val = imputer.transform(df_val.values)
df_val = pd.DataFrame(df_val, columns=columns)
df_val = df_val.groupby(["pid"], as_index=False).mean()

In [158]:
df_train.to_csv("df_train_philip.csv")
df_val.to_csv("df_val_philip.csv")

## Data formatting

In [112]:
df_train_preprocessed = pd.read_csv("df_train_philip.csv")
df_val_preprocessed = pd.read_csv("df_val_philip.csv")
df_train_preprocessed = df_train_preprocessed.sort_values(by=["pid"])
df_val_preprocessed = df_val_preprocessed.sort_values(by=["pid"])
df_train_label = df_train_label.sort_values(by=["pid"])

In [113]:
# Data formatting
X_train = df_train_preprocessed.drop(columns=IDENTIFIERS+['Unnamed: 0']).values
X_val = df_val_preprocessed.drop(columns=IDENTIFIERS+['Unnamed: 0']).values
# Create list with different label for each medical test
print("Creating a list of labels for each medical test")
y_train_medical_tests = []
for test in MEDICAL_TESTS:
    y_train_medical_tests.append(df_train_label[test].astype(int).values)

# Create list with different label for sepsis
print("Creating a list of labels for sepsis")
y_train_sepsis = []
for sepsis in SEPSIS:
    y_train_sepsis.append(df_train_label[sepsis].astype(int).values)

# Create list with different label for each vital sign
print("Creating a list of labels for each vital sign")
y_train_vital_signs = []
for sign in VITAL_SIGNS:
    y_train_vital_signs.append(df_train_label[sign].astype(int).values)

Creating a list of labels for each medical test
Creating a list of labels for sepsis
Creating a list of labels for each vital sign


## Scaler

In [114]:
# Scale data 
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

## Modelling medical tests

In [115]:
# # Modelling of medical tests using logistic regression with cross validation
# models = []
# losses = []
# columns_medical_tests = []
# for i, test in enumerate(MEDICAL_TESTS):
#     print(f"Fitting model for {test}.")

#     print("Applying feature selection")
#     feature_selector = SelectKBest(score_func=f_classif, k=3)
#     X_train = feature_selector.fit_transform(X_train, y_train_medical_tests[i])
#     X_test = feature_selector.transform(X_test)
#     columns = feature_selector.get_support(indices=True)
#     columns_medical_tests.append(columns)

#     print("Fitting model")
#     clf = LogisticRegressionCV(cv=5, random_state=42).fit(X_train, y_train_medical_tests[i])
#     models.append(clf)
#     print(roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]))
#     print(f"Finished test for medical tests.")

In [116]:
print("READY TO ROLL")

READY TO ROLL


In [117]:
def convert_to_cuda_tensor(X_train, X_test, y_train, y_test, device):
    """Converts a number of np.ndarrays to tensors placed on the device specified.

    Args:
        X_train (np.ndarray): (n_samples, n_features) array containing training features
        X_test (np.ndarray): (n_samples, n_features) array containing testing features
        y_train (np.ndarray): (n_samples,) array containing training labels
        y_test (np.ndarray): (n_samples,) array containing testing labels
        device (torch.device): device on which the tensors should be placed (CPU/CUDA GPU)

    Returns:

    """
    return (
        torch.from_numpy(X_train).to(device).float(),
        torch.from_numpy(X_test).to(device).float(),
        torch.from_numpy(y_train).to(device).float(),
        torch.from_numpy(y_test).to(device).float(),
    )


class Feedforward(torch.nn.Module):
    """ Definition of the feedfoward neural network. It currently has three layers which can be
    modified in the function where the network is trained.
    """

    def __init__(self, input_size, hidden_size, output_size, subtask, p=0.2):
        super(Feedforward, self).__init__()
        self.subtask = subtask
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout = torch.nn.Dropout(p=p)
        self.bn = torch.nn.BatchNorm1d(hidden_size)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        torch.nn.init.xavier_normal_(self.fc1.weight)
        self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
        torch.nn.init.xavier_normal_(self.fc2.weight)
        self.fcout = torch.nn.Linear(self.hidden_size, self.output_size)
        torch.nn.init.xavier_normal_(self.fcout.weight)

    def forward(self, x):
        """Function where the forward pass is defined. The backward pass is deternmined by the
            autograd function built into PyTorch.
        Args:
            x (torch.Tensor): Tensor (n_samples,n_features) tensor containing training input
                features
            subtask (int): subtask performed (choice: 1,2,3)
        Returns:
            output (torch.Tensor): (n_samples,n_features) tensor containing
                the predicted output for each sample.
        """
        assert self.subtask in [1, 2, 3]
        hidden = self.fc1(x)
        hidden_bn = self.bn(hidden)
        relu = self.sigmoid(hidden_bn)
        hidden = self.dropout(self.fc2(relu))
        hidden_bn = self.bn(hidden)
        relu = self.sigmoid(hidden_bn)
        output = self.fcout(relu)
        if self.subtask == 1 or self.subtask == 2:
            output = self.sigmoid(output)
        else:
            output = self.relu(output)
        return output


class Data(Dataset):
    """ Class used to load the data in minibatches to control the neural network stability during
        training.
    """

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.len = self.x.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.len

In [159]:
# # Modelling using extreme gradient boosting
# models = []
# losses = []
# feature_selector_medical_tests = []
# for i, test in enumerate(MEDICAL_TESTS):
#     print(f"Fitting model for {test}.")
#     X_train, X_test, y_train, y_test = train_test_split(
#     X_train_scaled, y_train_medical_tests[i], test_size=0.10, random_state=42, shuffle=True
#     )
#     # Coarse parameter grid not optimized at all yet
    
#     print("Resampling")
#     sampler = RandomUnderSampler(random_state=42)
#     X_train_res, y_train_res = sampler.fit_resample(X_train, y_train)
    
#     print("Applying feature selection")
#     feature_selector = SelectKBest(score_func=f_classif, k=10)
#     X_train = feature_selector.fit_transform(X_train_res, y_train_res)
#     X_test = feature_selector.transform(X_test)
#     feature_selector_medical_tests.append(feature_selector)
    
    
#     X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = convert_to_cuda_tensor(
#             X_train, X_test, y_train_res, y_test, DEVICE
#     )
    
#     model = Feedforward(
#             input_size=X_train_tensor.shape[1],
#             hidden_size=100,
#             output_size=1,
#             subtask=1,
#             p=0.1,
#     )
    
#     criterion = torch.nn.BCEWithLogitsLoss()

# #     if optim == "SGD":
# #         optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# #     elif optim == "Adam":
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#     dataset = Data(X_train_tensor, y_train_tensor)
#     batch_size = 1024  # Ideally we want powers of 2
#     trainloader = DataLoader(dataset=dataset, batch_size=batch_size)

#     if torch.cuda.is_available():
#         model.cuda()
#     model.float()
    
    
#     dirpath = os.path.join(os.getcwd(), "runs")
#     fileList = os.listdir(dirpath)
#     for fileName in fileList:
#         shutil.rmtree(dirpath + "/" + fileName)
    
#     now = time.strftime("%Y%m%d-%H%M%S")
#     writer = SummaryWriter(
#         log_dir=f"runs/ann_network_runs_{200}_epochs_{now}_{test}"
#     )
#     for epoch in list(range(500)):
#         LOSS = []
#         for x, y in trainloader:
#             yhat = model(x)
#             loss = criterion(yhat.float(), y.reshape((y.shape[0], 1)))
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             LOSS.append(loss)
#         X_test_tensor = X_test_tensor.to(DEVICE)
#         y_test_pred = model(X_test_tensor).cpu().detach().numpy()
#         y_train_pred = model(X_train_tensor).cpu().detach().numpy()
#         loss_average = sum(LOSS) / len(LOSS)
#         writer.add_scalar("Training_loss", loss_average, epoch)
#         ROC_train = roc_auc_score(y_train_res, y_train_pred)
#         ROC_test = roc_auc_score(y_test, y_test_pred)
# #         writer.add_scalar("ROC train", ROC_train, epoch)
# #         writer.add_scalar("ROC test", ROC_test, epoch)
#         print(f"Epoch: {epoch} - ROC train: {ROC_train} - ROC test: {ROC_test}", end="\r")
#     print("")
        
# #     models.append(model)
# #     model.eval()
# #     print(f"ROC score on test set {roc_auc_score(y_test, model(X_test_tensor))}")
# #     print(f"CV score {coarse_search.best_score_}")
# print(f"Finished test for medical tests.")

In [166]:
# Modelling using extreme gradient boosting
clf = xgb.XGBClassifier(objective="binary:logistic", n_thread=-1)
models = []
losses = []
feature_selector_medical_tests = []
for i, test in enumerate(MEDICAL_TESTS):
    print(f"Fitting model for {test}.")
    X_train, X_test, y_train, y_test = train_test_split(
    X_train_scaled, y_train_medical_tests[i], test_size=0.10, random_state=42, shuffle=True
    )
    
    print("Resampling")
    sampler = RandomUnderSampler(random_state=42)
    X_train_res, y_train_res = sampler.fit_resample(X_train, y_train)
    
    print("Applying feature selection")
    feature_selector = SelectKBest(score_func=f_classif, k=5)
    X_train_selected = feature_selector.fit_transform(X_train_res, y_train_res)
    X_test = feature_selector.transform(X_test)
    feature_selector_medical_tests.append(feature_selector)
    
    print("Fitting coarse model")
    # Coarse parameter grid not optimized at all yet
    coarse_param_grid = {
        "booster": ["dart"],
        "eta": np.arange(0,1,0.1),
        "min_child_weight": range(1, 10, 1),
        "max_depth": range(4, 10, 1),
        "gamma": range(0, 100, 1),
        "max_delta_step": range(1, 10, 1),
        "subsample": np.arange(0.1, 1, 0.05),
        "colsample_bytree": np.arange(0.3, 1, 0.05),
        "n_estimators": range(50, 150, 1),
        "scale_pos_weight": [1],
        "reg_lambda": [0, 1], # Ridge regularization
        "reg_alpha": [0, 1], # Lasso regularization
        "eval_metric": ["error"],
        "verbosity": [1]
    }
    coarse_search = RandomizedSearchCV(estimator=clf,
            param_distributions=coarse_param_grid, scoring="roc_auc",
            n_jobs=-1, cv=10, n_iter=1000, verbose=1)
    coarse_search.fit(X_train_selected, y_train_res)
    print(coarse_search.best_estimator_.predict_proba(X_test)[:,1])
    print(f"ROC score on test set {roc_auc_score(y_test, coarse_search.best_estimator_.predict_proba(X_test)[:,1])}")
    print(f"CV score {coarse_search.best_score_}")
    best_params = coarse_search.best_params_
#     print(best_params)
#     print("Fitting fine model")
    # Fine parameter grid not optimized at all yet
#     fine_param_grid = {
#         "booster": best_params["booster"],
#         "eta": np.arange(best_params["eta"]-0.05, best_params["eta"]+0.05, 0.003),
#         "min_child_weight": [best_params["min_child_weight"]],
#         "max_depth": [best_params["max_depth"]],
#         "gamma": np.arange(best_params["gamma"]-3, best_params["gamma"]+3, 0.05),
#         "max_delta_step": [best_params["max_delta_step"]],
#         "subsample": np.arange(best_params["subsample"]-0.05, best_params["subsample"]+0.05, 0.002),
#         "colsample_bytree": [best_params["colsample_bytree"]],
#         "n_estimators": range(best_params["n_estimators"]-10, best_params["n_estimators"]+10, 1),
#         "scale_pos_weight": [1],
#         "reg_lambda": [best_params["reg_lambda"]], # Ridge regularization
#         "reg_alpha": [best_params["reg_alpha"]], # Lasso regularization
#         "eval_metric": ["error"],
#         "verbosity": [2]
#     }
#     fine_search = RandomizedSearchCV(estimator=clf,
#             param_distributions=fine_param_grid, scoring="roc_auc",
#             n_jobs=-1, cv=10, n_iter=10, verbose=1)
#     fine_search.fit(X_train_selected, y_train_res)
#     print(coarse_search.best_estimator_.predict_proba(X_test)[:,1])
    print(f"ROC score on test set {roc_auc_score(y_test, coarse_search.best_estimator_.predict_proba(X_test)[:,1])}")
    print(f"CV score {coarse_search.best_score_}")
    
    
    
    models.append(coarse_search.best_estimator_)
    
print(f"Finished test for medical tests.")

Fitting model for LABEL_BaseExcess.
Resampling
Applying feature selection
Fitting coarse model
Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed: 17.1min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed: 20.1min
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed: 23.9min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed: 28.2min
[Parallel(n_jobs=-1)]: Done 10000 out of 1000

[0.92124844 0.11344181 0.29810002 ... 0.91302997 0.27503392 0.8153258 ]
ROC score on test set 0.879664081103796
CV score 0.8627901854812372
ROC score on test set 0.879664081103796
CV score 0.8627901854812372
Fitting model for LABEL_Fibrinogen.
Resampling
Applying feature selection
Fitting coarse model
Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:   47.0s
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 10000 out of 1000

[0.718125   0.5770305  0.52418727 ... 0.4888774  0.32086784 0.36808202]
ROC score on test set 0.7243470422535212
CV score 0.7329693651574803
ROC score on test set 0.7243470422535212
CV score 0.7329693651574803
Fitting model for LABEL_AST.
Resampling
Applying feature selection
Fitting coarse model
Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 1528 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done 2428 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Done 3072 tasks      | elapsed:   48.3s
[Parallel(n_jobs=-1)]: Done 4132 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 5632 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 7332 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 9232 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:  2.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[0.8458845  0.62316173 0.28457513 ... 0.4416283  0.3965993  0.6129491 ]
ROC score on test set 0.7264207650273226
CV score 0.7335745216034641
ROC score on test set 0.7264207650273226
CV score 0.7335745216034641
Fitting model for LABEL_EtCO2.
Resampling
Applying feature selection
Fitting coarse model
Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 498 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 848 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done 1298 tasks      | elapsed:   48.5s
[Parallel(n_jobs=-1)]: Done 1848 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2498 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 3248 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 4098 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 5048 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 6098 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 7248 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 8498 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 9848 tasks      | elapsed:  5.7min


[0.62657565 0.3886174  0.32917774 ... 0.61635196 0.25950286 0.5672045 ]
ROC score on test set 0.8425446428571428
CV score 0.801199280844551
ROC score on test set 0.8425446428571428
CV score 0.801199280844551
Finished test for medical tests.


[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:  5.8min finished


In [169]:
import joblib
for i, model in enumerate(models):
    joblib.dump(models[i], f"xgboost_fine_{MEDICAL_TESTS[i]}.pkl")

In [171]:
val_pids = np.unique(df_val["pid"].values)

In [172]:
# Get predictions for medical tests
df_pred_medical_test = pd.DataFrame(index=val_pids, columns=MEDICAL_TESTS)
for i, test in enumerate(MEDICAL_TESTS):
    feature_selector = feature_selector_medical_tests[i]
    X_val_vital_sign = feature_selector.transform(X_val_scaled)
    model_for_test = models[i]
#     print(model_for_test.predict_proba(X_val_vital_sign))
    y_pred = model_for_test.predict_proba(X_val_vital_sign)[:, 1]
    df_pred_medical_test[test] = y_pred

df_pred_medical_test = df_pred_medical_test.reset_index().rename(columns={"index": "pid"})

In [173]:
df_pred_medical_test

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2
0,0.0,0.832393,0.429798,0.896758,0.961712,0.913878,0.849364,0.150986,0.603449,0.647068,0.434431
1,3.0,0.131074,0.304217,0.463766,0.480553,0.439553,0.238957,0.698675,0.274292,0.446310,0.232469
2,5.0,0.417147,0.304217,0.443913,0.433482,0.425904,0.277688,0.473744,0.404472,0.446310,0.218275
3,7.0,0.959387,0.911051,0.944069,0.961004,0.867512,0.672507,0.352497,0.647528,0.911838,0.269971
4,9.0,0.384726,0.298570,0.566566,0.562898,0.542512,0.396905,0.221129,0.206536,0.570115,0.324562
...,...,...,...,...,...,...,...,...,...,...,...
12659,31647.0,0.338738,0.304217,0.406793,0.373973,0.411888,0.300359,0.094801,0.290623,0.446310,0.232469
12660,31649.0,0.175901,0.608352,0.760341,0.811736,0.611820,0.340112,0.677199,0.271969,0.739103,0.281824
12661,31651.0,0.929130,0.325646,0.474042,0.461187,0.443320,0.697765,0.273953,0.724276,0.364428,0.437264
12662,31652.0,0.082532,0.317979,0.525401,0.593110,0.526376,0.207084,0.635182,0.299705,0.243108,0.341546


## Modelling sepsis

In [174]:
# Model and predict sepsis

clf = xgb.XGBClassifier(objective="binary:logistic", n_thread=-1)


X_train, X_test, y_train, y_test = train_test_split(
    X_train_scaled, y_train_sepsis[0], test_size=0.10, random_state=42, shuffle=True
)


param_grid = {
        "booster": ["dart"],
        "eta": np.arange(0,1,0.1),
        "min_child_weight": range(1, 10, 1),
        "max_depth": range(4, 10, 1),
        "gamma": range(0, 100, 1),
        "max_delta_step": range(1, 10, 1),
        "subsample": np.arange(0.1, 1, 0.05),
        "colsample_bytree": np.arange(0.3, 1, 0.05),
        "n_estimators": range(50, 150, 1),
        "scale_pos_weight": [1],
        "reg_lambda": [0, 1], # Ridge regularization
        "reg_alpha": [0, 1], # Lasso regularization
        "eval_metric": ["error"],
        "verbosity": [1]
    }

print("Resampling")
sampler = RandomUnderSampler()
X_train_res, y_train_res = sampler.fit_resample(X_train, y_train)

print("Applying feature selection")
feature_selector = SelectKBest(score_func=f_classif, k=30)
X_train = feature_selector.fit_transform(X_train_res, y_train_res)
X_test = feature_selector.transform(X_test)


print("Fitting model")
coarse_search = RandomizedSearchCV(estimator=clf,
        param_distributions=param_grid, scoring="roc_auc",
        n_jobs=-1, cv=10, n_iter=1000, verbose=1)
print(y_train_res)
coarse_search.fit(X_train, y_train_res)

sepsis_model = coarse_search.best_estimator_
print(f"ROC score on test set {roc_auc_score(y_test, coarse_search.best_estimator_.predict_proba(X_test)[:,1])}")
print(f"CV score {coarse_search.best_score_}")
print(f"Finished test for medical tests.")

Resampling
Applying feature selection
Fitting model
[0 0 0 ... 1 1 1]
Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 10000 out of 1000

ROC score on test set 0.7539753938168123
CV score 0.7092163131313131
Finished test for medical tests.


In [176]:
joblib.dump(sepsis_model, f"xgboost_fine_sepsis.pkl")

['xgboost_fine_sepsis.pkl']

In [177]:
X_val_sepsis = feature_selector.transform(X_val_scaled)
y_pred = sepsis_model.predict_proba(X_val_sepsis)[:,1]
df_pred_sepsis = pd.DataFrame(y_pred, index=val_pids, columns=SEPSIS)
df_pred_sepsis = df_pred_sepsis.reset_index().rename(columns={"index": "pid"})

## Modelling vital signs

In [180]:
# Modelling of vital signs
models = []
losses = []
feature_selectors_vital_signs = []
clf = xgb.XGBRegressor(objective="reg:squarederror", n_thread=-1)

for i, sign in enumerate(VITAL_SIGNS):
    print(f"Fitting model for {sign}.")
    X_train, X_test, y_train, y_test = train_test_split(
        X_train_scaled, y_train_vital_signs[i], test_size=0.10, random_state=42, shuffle=True
    )

    print("Applying feature selection")
    feature_selector = SelectKBest(score_func=f_classif, k=5)
    X_train_selected = feature_selector.fit_transform(X_train, y_train)
    X_test_selected = feature_selector.transform(X_test)
    feature_selectors_vital_signs.append(feature_selector)

    print("Fitting model")
    
    param_grid = {
        "booster": ["dart"],
        "eta": np.arange(0,1,0.1),
        "min_child_weight": range(1, 10, 1),
        "max_depth": range(4, 10, 1),
        "gamma": range(0, 100, 1),
        "max_delta_step": range(1, 10, 1),
        "subsample": np.arange(0.1, 1, 0.05),
        "colsample_bytree": np.arange(0.3, 1, 0.05),
        "n_estimators": range(50, 150, 1),
        "scale_pos_weight": [1],
        "reg_lambda": [0, 1], # Ridge regularization
        "reg_alpha": [0, 1], # Lasso regularization
        "eval_metric": ["error"],
        "verbosity": [1]
    }


    
    coarse_search = RandomizedSearchCV(estimator=clf,
            param_distributions=param_grid, scoring="r2",
            n_jobs=-1, cv=10, n_iter=100, verbose=1)
    coarse_search.fit(X_train_selected, y_train)
    models.append(coarse_search.best_estimator_)
    print(f"CV score {coarse_search.best_score_}")
    print(f"Test score is {r2_score(y_test, coarse_search.best_estimator_.predict(X_test_selected))}")
    print(f"Finished test for medical tests.")

Fitting model for LABEL_RRate.
Applying feature selection
Fitting model
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  8.3min finished


CV score 0.3762834615674595
Test score is 0.3651876778962392
Finished test for medical tests.
Fitting model for LABEL_ABPm.
Applying feature selection
Fitting model
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  6.7min finished


CV score 0.5772479021545291
Test score is 0.5894346373011152
Finished test for medical tests.
Fitting model for LABEL_SpO2.
Applying feature selection
Fitting model
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   49.5s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  4.5min finished


CV score 0.30896617018979444
Test score is 0.3392388962178333
Finished test for medical tests.
Fitting model for LABEL_Heartrate.
Applying feature selection
Fitting model
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  8.0min finished


CV score 0.5974856981684256
Test score is 0.6043524041430639
Finished test for medical tests.


In [181]:
for i, model in enumerate(models):
    joblib.dump(models[i], f"xgboost_fine_{VITAL_SIGNS[i]}.pkl")

In [182]:
# Get predictions for medical tests
df_pred_vital_signs = pd.DataFrame(index=val_pids, columns=VITAL_SIGNS)
for i, test in enumerate(VITAL_SIGNS):
    feature_selector = feature_selectors_vital_signs[i]
    X_val_vital_sign = feature_selector.transform(X_val_scaled)
    model_for_test = models[i]
    y_pred = model_for_test.predict(X_val_vital_sign)
    df_pred_vital_signs[test] = y_pred

df_pred_vital_signs = df_pred_vital_signs.reset_index().rename(columns={"index": "pid"})

## Export to ZIP file

In [183]:
df_predictions = pd.merge(df_pred_medical_test, df_pred_sepsis, on="pid")
df_predictions = pd.merge(df_predictions, df_pred_vital_signs, on="pid")
print("Export predictions DataFrame to a zip file")
print(df_predictions)
df_predictions.to_csv(
    "predictions.csv",
    index=None,
    sep=",",
    header=True,
    encoding="utf-8-sig",
    float_format="%.2f",
)

with zipfile.ZipFile("predictions.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write("predictions.csv")
os.remove("predictions.csv")

Export predictions DataFrame to a zip file
           pid  LABEL_BaseExcess  LABEL_Fibrinogen  LABEL_AST  \
0          0.0          0.832393          0.429798   0.896758   
1          3.0          0.131074          0.304217   0.463766   
2          5.0          0.417147          0.304217   0.443913   
3          7.0          0.959387          0.911051   0.944069   
4          9.0          0.384726          0.298570   0.566566   
...        ...               ...               ...        ...   
12659  31647.0          0.338738          0.304217   0.406793   
12660  31649.0          0.175901          0.608352   0.760341   
12661  31651.0          0.929130          0.325646   0.474042   
12662  31652.0          0.082532          0.317979   0.525401   
12663  31655.0          0.117321          0.323369   0.573651   

       LABEL_Alkalinephos  LABEL_Bilirubin_total  LABEL_Lactate  \
0                0.961712               0.913878       0.849364   
1                0.480553               0.